## Frogtown DataLoad Workbook 05/06/18

### by Abu Nayeem 

This is just clean coded process going over the uploading steps assuming all the preliminary steps after decoder are made


## Table of contents <a name="content"></a>
* [Data Setup](#setup)
* [Create Variables](#create)
* [Intersection Table: Preparation](#intersection)
* [Address Table: Preparation](#address)
* [Preliminary Data Cleaning](#pre)


### Data Setup <a name="setup"></a>

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import HTML
from IPython.display import display
import requests # library to handle requests

#Load Data
df_crime = pd.read_csv('Datasets/Crime_Incident_Report_-_Dataset.csv')

#rename columns
cols= ['Case','Date','Time','Code','IncType','Incident','Grid','NNum','Neighborhood','Block','CallDispCode','CallDisposition', 'Count']
df_crime.columns= cols

#selection for Frogtown and nearby area
df=df_crime.query('Grid in [66,67, 68, 86, 87,88,89, 90, 91, 92,106,107,108,109,110]')

df.head(4)

,Case,Date,Time,Code,IncType,Incident,Grid,NNum,Neighborhood,Block,CallDispCode,CallDisposition,Count
17,19088980,04/30/2019,2019-04-30T22:48:00.000,2619,"Weapons, Discharging a Firearm in the City Limits",Discharge,90.0,7,7 - Thomas/Dale(Frogtown),19X SHERBURNE AV,RR,Report Written,1
24,19088940,04/30/2019,2019-04-30T22:06:00.000,9954,Proactive Police Visit,Proactive Police Visit,109.0,8,8 - Summit/University,43X DALE ST N,A,Advised,1
42,19088880,04/30/2019,2019-04-30T20:30:00.000,710,"Motor Vehicle Theft, Automobile",Auto Theft,89.0,7,7 - Thomas/Dale(Frogtown),59X THOMAS AV,RR,Report Written,1
52,19089126,04/30/2019,2019-04-30T20:00:00.000,693,"Theft, All Other, Over $1000",Theft,110.0,8,8 - Summit/University,24X AURORA AV,RR,Report Written,1


### Create New Variables <a name="create"></a>

In [2]:
#Add Time Variables
df= df[df.Case != 18254093] #messed up time variable

#Convert Date to Datetime!
from datetime import datetime

df['DateTime']= pd.to_datetime(df['Date']) # Create new column called DateTime
df['Year']= df['DateTime'].dt.year #create year column
df['DayofWeek']=df['DateTime'].dt.dayofweek #create day of the week column where default 0=Monday
df['Weekend'] = df['DayofWeek'].apply(lambda x: 1 if (x>4)  else 0) #Create a weekend category
df['Month'] = df['DateTime'].dt.month # Create Month Category
df['Day'] = df['DateTime'].dt.day #Create Day of the Current month
df['DayYear'] = df['DateTime'].dt.dayofyear  #Create Day of the year (0-365)
df['Day_Max'] = df.iloc[0,-1] #selects uptodate day; NOTE: the data is sorted chronologically

#Hour Data
df['TimeHour']= pd.to_datetime(df['Time'])
df['Hour'] = df['TimeHour'].dt.hour.astype(int) #Create Hour Colum
df['LateNight'] = df['Hour'].apply(lambda x: 1 if (x>21 or x<5)  else 0) #Latenight designation from 10Pm to 6PM

#Creating the intersection Column. Note: the Block column has the address information
df.Block = df.Block.astype(str) #first change the type to string
df['Block']= df['Block'].str.lower() #lowercase string to create uniformity

#While scanning the data I noticed that all intersections had "&" 
df['Intersection'] = df['Block'].apply(lambda x: 1 if '&' in x else 0) #intersection

df.head(5)

,Case,Date,Time,Code,IncType,Incident,Grid,NNum,Neighborhood,Block,...,DayofWeek,Weekend,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection
17,19088980,04/30/2019,2019-04-30T22:48:00.000,2619,"Weapons, Discharging a Firearm in the City Limits",Discharge,90.0,7,7 - Thomas/Dale(Frogtown),19x sherburne av,...,1,0,4,30,120,120,2019-04-30 22:48:00,22,1,0
24,19088940,04/30/2019,2019-04-30T22:06:00.000,9954,Proactive Police Visit,Proactive Police Visit,109.0,8,8 - Summit/University,43x dale st n,...,1,0,4,30,120,120,2019-04-30 22:06:00,22,1,0
42,19088880,04/30/2019,2019-04-30T20:30:00.000,710,"Motor Vehicle Theft, Automobile",Auto Theft,89.0,7,7 - Thomas/Dale(Frogtown),59x thomas av,...,1,0,4,30,120,120,2019-04-30 20:30:00,20,0,0
52,19089126,04/30/2019,2019-04-30T20:00:00.000,693,"Theft, All Other, Over $1000",Theft,110.0,8,8 - Summit/University,24x aurora av,...,1,0,4,30,120,120,2019-04-30 20:00:00,20,0,0
66,19088803,04/30/2019,2019-04-30T18:40:32.000,9954,Proactive Police Visit,Proactive Police Visit,90.0,7,7 - Thomas/Dale(Frogtown),17x charles av,...,1,0,4,30,120,120,2019-04-30 18:40:32,18,0,0


### Prepare Intersection table <a name="intersection"></a>

In [4]:
#Load clean intersections key
df_Intersection_key = pd.read_csv('Intersection_key_clean.csv')

# Create a new dateframe specifying only intersections from primary dataset
dfI=df.query('Intersection ==1')
print('The intersection table dimension are ' + str(dfI.shape))
#print(dfI.Block.head(10))


#Split the strings
new=dfI['Block'].str.split("& ", n = 1, expand = True) 
dfI['Inter2']= new[1]
new=dfI['Block'].str.split(" ", n = 1, expand = True) #Note the code specifies the first time a space occured
dfI['Inter1']=new[0]

#Create the IndexKey; recall we prepared the IntersectionKey where it considers any order
dfI['IndexKey']= dfI['Inter1']+ '_' + dfI['Inter2']
dfI.reset_index()
dfI=pd.merge(dfI, df_Intersection_key, on='IndexKey', how='left')



The intersection table dimension are (4948, 25)


In [5]:
#Drop rows with missing coordinates
dfI=dfI[dfI['Coordinates'].notnull()]

# Separate Latitude and Longitude 
new=dfI['Coordinates'].str.split(",", n = 1, expand = True) 
# making seperate first name column from new data frame 
dfI['Latitude']= pd.to_numeric(new[0]) #pd.to_numeric convert it to float
dfI['Longitude']= pd.to_numeric(new[1])

#Renaming columns
dfI['Block']=dfI['OutputKey'] #for practical purposes it makes sense
Drop_col=['Inter2','Inter1', 'IndexKey', 'Coordinates', 'OutputKey']
dfI_Final=dfI.drop(Drop_col, axis=1,)
dfI_Final.head(5)


,Case,Date,Time,Code,IncType,Incident,Grid,NNum,Neighborhood,Block,...,Month,Day,DayYear,Day_Max,TimeHour,Hour,LateNight,Intersection,Latitude,Longitude
0,19088395,04/30/2019,2019-04-30T08:00:00.000,861,"Assault, Domestic, Opposite Sex",Simple Asasult Dom.,89.0,7,7 - Thomas/Dale(Frogtown),dale_edmund,...,4,30,120,120,2019-04-30 08:00:00,8,0,1,44.958439,-93.126376
1,19088075,04/29/2019,2019-04-29T21:40:24.000,9954,Proactive Police Visit,Proactive Police Visit,87.0,7,7 - Thomas/Dale(Frogtown),milton_thomas,...,4,29,119,120,2019-04-29 21:40:24,21,0,1,44.959361,-93.139031
2,19088071,04/29/2019,2019-04-29T21:39:13.000,9954,Proactive Police Visit,Proactive Police Visit,88.0,7,7 - Thomas/Dale(Frogtown),stalbans_thomas,...,4,29,119,120,2019-04-29 21:39:13,21,0,1,44.959350,-93.128908
4,19087852,04/29/2019,2019-04-29T16:56:01.000,9954,Proactive Police Visit,Proactive Police Visit,89.0,7,7 - Thomas/Dale(Frogtown),arundel_university,...,4,29,119,120,2019-04-29 16:56:01,16,0,1,44.955853,-93.118679
5,19087606,04/29/2019,2019-04-29T10:52:31.000,9954,Proactive Police Visit,Proactive Police Visit,66.0,11,11 - Hamline/Midway,syndicate_hubbard,...,4,29,119,120,2019-04-29 10:52:31,10,0,1,44.965660,-93.154277


### Prepare Address Decoder Table <a name="address"></a>

It is so remarkably short

In [6]:
#Split Data
dfW=df.query('Intersection==0')

#Load Complete Decoder Key
df_C= pd.read_csv('SemiKey.csv')
df_C= df_C[['Block','Latitude','Longitude']]

# Merge with the dataset and remove missing values
dC=pd.merge(dfW, df_C, on='Block', how='left')
dC=dC.fillna('Mi')
dC=dC.query('Latitude != "Mi"')

#Bringing the data back together 
fg= dfI.append(dC, ignore_index=True)

### Preliminary Pre-Cleaning steps <a name="clean"></a>

It is good to do the data pre-cleaning steps here to reduce clutter on a working notebook. A few edits include renaming some values, clustering crimes together and creating some dummies. It is saved in a csv, which is used for execution

In [7]:
#Few Quick Edits
fg.CallDisposition.loc[(fg['CallDisposition'] == 'G - Gone on Arrival')] = 'Gone on Arrival'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'A - Advised')] = 'Advised'
fg.CallDisposition.loc[(fg['CallDisposition'] == 'RR - Report Written')] = 'Report Written'
fg.Incident.loc[(fg['Incident'] == 'Simple Asasult Dom.')] = 'Simple Assault Dom.'
fg.Incident.loc[(fg['Incident'] == 'Graffiti')] = 'Vandalism'
fg.Incident.loc[fg["Incident"].isin([ "Rape","Agg. Assault",'Homicide'])]= 'Violent'
fg.Incident.loc[fg["Incident"].isin(["Simple Assault Dom.","Agg. Assault Dom."])]= 'Domestic Assault'

#[fg["Incident"].isin(["Simple Assault Dom.", "Rape"])


#Create a dummy for each crime category
fg= pd.concat([fg,pd.get_dummies(fg['Incident'])], axis=1)
fg= pd.concat([fg,pd.get_dummies(fg['CallDisposition'])], axis=1)

fg.to_csv('FGCrime_Final.csv', encoding='utf-8', index=False)